In [3]:
import os 
from pathlib import Path
from eth_account import Account
import sys
path = os.path.abspath('')
parent_dir = str(Path(path).parents[2])
main_dir = str(Path(path).parents[2])
sys.path.append(parent_dir)
from pyledger.extras.evmnet.participant_scripts import *
from pyledger.extras.evmnet.evmpadl import EvmLedger
from pyledger.ledger import BankCommunication
from pyledger.examples.padl_deployERC import get_url, get_issuer_account_dict

## SETUP: Make connection, Issuer deploys contracts, another participants arrives, shares address and public key.

In [5]:
url = get_url()
chain_id = 1337
local_dirname = str(Path(path).parents[2])
w3 = Web3(Web3.HTTPProvider(url, request_kwargs={'timeout': 600000}))

####################################################################################################################
issuer_account_dict = get_issuer_account_dict()
account_address = issuer_account_dict['address']
example_private_key = issuer_account_dict['private_key']
public_key = issuer_account_dict['public_key']
private_key = example_private_key
account_address2 = "0x8c171946c84D214e9F54fF22F515347dA4d78F7B" # participants account address
account_address = Web3.toChecksumAddress(account_address)
account_address2 = Web3.toChecksumAddress(account_address2)
LocalAccount = Account.from_key(example_private_key)

#####################################################################################################################
ledger,bank = deploy_PADLOnChain(example_private_key, contract_tx_name="PADLOnChainBN", file_name_contract="PADLOnChainBN.sol")
contract_address = ledger.deployed_address
account_dict = create_account(contract_address=contract_address)
add_participant(account_dict["address"])#, "Issuer 0", contract_tx_name=contract_tx_name, file_name_contract=file_name_contract)
publickey_bank = account_dict['public_key']

#####################################################################################################################
bank=register_padl('Bank', account_dict, v0=[0,0] , types={'0': 'cash-token', '1': 'fund-token'})
bank_gkp = PadlEVM(secret_key=account_dict['private_key'], contract_address=account_dict['contract_address'], contract_tx_name="PADLOnChainBN", file_name_contract="PADLOnChainBN.sol")
pks = bank_gkp.retrieve_pks()
pub_keys = [public_key, publickey_bank]


deploy
created ledger
added participant to ledger
new account created with address 0x1A672dF1384e7F6c065e8667021049b20572Ff83
address 0x1A672dF1384e7F6c065e8667021049b20572Ff83
account address 0xFE3B557E8Fb62b89F4916B721be55cEb828dBd73


## To deploy private token we start with initial supply encrypted as commitment and token and address of contract with verification schemes for zkproofs

In [6]:
# Generate initial supply commitment token
initSupply = 100
r = r_blend()
cm = Commit(zkbp.gen_GH(), initSupply, r).eval.get
tk = zkbp.to_token_from_pk(public_key, r.val).get
commit = BNCurve.get_xy(cm)
token = BNCurve.get_xy(tk)

In [7]:
contract_args = (100, commit, token, contract_address, ledger.bnaddress, ledger.eqaddress, ledger.consaddress)
ev = EvmLedger(BankCommunication(), "PadlTokenBN.sol", local_dirname, w3, chain_id, account_address, private_key, "PadlTokenBN")
deployed_address = ev.deploy_padl_erc(contract_args)

## ERC contract deployed, check private balance

In [8]:
conn = ev.connect_to_evm()
contract = conn['contract_obj']
b = contract.functions.privateBalanceOf(account_address).call()
print(b)
check_balance_by_commit_token('Issuer 0', (b[0], b[1]), (b[2], b[3]))

[11144450123644420088300977058589705499228572341681427418602605436952194929011, 6375695031510797692084216548309518758817063188403536718930574482292251350322, 19020900772094321149345331822032156566737754254110884207605591971457462817099, 13759308080568389150176506272352638813954110926142468533084424804277266825928]
balance is 100


100

In [9]:
print(contract.functions.privateBalanceOf(account_address2).call())

[0, 0, 0, 0]


## Next generate a transaction including commitment, token and all zk proofs and call privateTransfer function on token contract

In [10]:
tx_str = get_private_tx_str(pub_keys, [[-2, 2]], file_name='Issuer 0', state=[cm,tk], old_balance=100, audit_pk=None)
nonce = w3.eth.get_transaction_count(account_address)
transaction = contract.functions.privateTransfer(account_address, account_address2, tx_str[0], tx_str[1]).buildTransaction(
    {
        "chainId":chain_id, "from":account_address, "nonce":nonce
    }
)

signed_txn = w3.eth.account.sign_transaction(transaction,private_key=private_key)
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

b = contract.functions.privateBalanceOf(account_address).call()
check_balance_by_commit_token('Issuer 0', (b[0], b[1]), (b[2], b[3]))
b = contract.functions.privateBalanceOf(account_address2).call()
check_balance_by_commit_token('Bank 1', (b[0], b[1]), (b[2], b[3]))
print('transaction done')

balance is 98
balance is 2
transaction done


In [11]:
check_balance_by_commit_token('Issuer 0', (b[0], b[1]), (b[2], b[3]))

balance is -1


-1

In [12]:
b = contract.functions.privateBalanceOf(account_address).call()
bank = utils.load_bank_from_file("Issuer 0")
cc = zkbp.from_str(BNCurve.get_compressed_ecpoint(b[0], b[1]))
tc = zkbp.to_token_from_str(BNCurve.get_compressed_ecpoint(b[2], b[3]))
gh = zkbp.gen_GH()
bal = zkbp.get_brut_v(cc,tc, gh, bank.sk_pk_obj, MAX)
print(bal) 

98


In [13]:
tx_str = get_private_tx_str(pub_keys, [[-2, 2]], file_name='Issuer 0', state=[cc.get,tc.get], old_balance=98, audit_pk=None)

In [14]:
nonce = w3.eth.get_transaction_count(account_address)
transaction = contract.functions.privateTransfer(account_address, account_address2, tx_str[0], tx_str[1]).buildTransaction(
    {
        "chainId":chain_id, "from":account_address, "nonce":nonce
    }
)

signed_txn = w3.eth.account.sign_transaction(transaction,private_key=private_key)
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)


In [15]:
b = contract.functions.privateBalanceOf(account_address).call()
check_balance_by_commit_token('Issuer 0', (b[0], b[1]), (b[2], b[3]))

balance is 96


96

In [16]:
b = contract.functions.privateBalanceOf(account_address2).call()
check_balance_by_commit_token('Bank 1', (b[0], b[1]), (b[2], b[3]))

balance is 4


4